## Data Cleaning

In this notebook, the raw NYC taxi dataset is being cleaned by:
- Removing invalid and corrupt records
- Handling missing values
- Filtering implausible trips
- Standardizing schema
- Validating key business constraints

The goal is to produce a modeling-ready dataset that aligns with the project’s data dictionary and analytical assumptions.


In [3]:
import pandas as pd

RAW_SNAPSHOT_PATH = "../../data/processed/raw_combined_2019_q1.parquet"
CLEAN_PATH = "../../data/processed/nyc_clean_2019_q1.parquet"

df = pd.read_parquet(RAW_SNAPSHOT_PATH)
print("Loaded raw snapshot:", df.shape)


Loaded raw snapshot: (22519712, 19)


In [4]:
# Create copy
df_clean = df.copy()

A working copy of the raw dataset is created to ensure reproducibility and preserve the original data for auditing and debugging.

In [5]:
# Filter data based on passenger count violations
df_clean = df_clean[df_clean["passenger_count"].between(1, 6)]

In [6]:
# Filter data based on trip distance violations
df_clean = df_clean[df_clean["trip_distance"] > 0]

In [7]:
# Filter data based on fare and total amount violations
df_clean = df_clean[
    (df_clean["fare_amount"] > 0) &
    (df_clean["total_amount"] > 0)
]

In [8]:
# Calculate trip duration in minutes
df_clean["trip_duration_min"] = (
    (df_clean["tpep_dropoff_datetime"] - df_clean["tpep_pickup_datetime"])
    .dt.total_seconds() / 60
)

# Filter data based on trip duration violations
df_clean = df_clean[
    (df_clean["trip_duration_min"] > 0) &
    (df_clean["trip_duration_min"] < 240)
]

### Validity Filtering

We remove trips that violate basic physical and business constraints:

- Passenger count outside [1, 6]
- Non-positive trip distance
- Non-positive fare or total amount
- Negative or extremely long trip durations (> 4 hours)

These records are treated as corrupt and excluded from analysis.

In [9]:
# Enforce time window
start_date = "2019-01-01"
end_date   = "2019-03-31 23:59:59"

df_clean = df_clean[
    (df_clean["tpep_pickup_datetime"] >= start_date) &
    (df_clean["tpep_pickup_datetime"] <= end_date)
]

### Time Window Enforcement

Some trips fall outside the intended January-March 2019 window.

These records are excluded to enforce a strict analytical slice and prevent time leakage.

In [10]:
# Handle missing congestion subcharge values
df_clean["congestion_surcharge"] = df_clean["congestion_surcharge"].fillna(0)

In [11]:
# Handle missing tip amount values
df_clean["tip_amount"] = df_clean["tip_amount"].fillna(0)

In [13]:
# Handle missing store and forward flag
df_clean["store_and_fwd_flag"] = df_clean["store_and_fwd_flag"].fillna("N")

### Missing Value Handling

- `congestion_surcharge` NAs are filled with 0s, as this was introduced later in 2019 and only applies to specific zones and conditions.
- `tip_amount` NAs are filled with 0s, as no-tip rides are valid.
- `store_and_fwd_flag` is filled with "N" if missing.

No rows are dropped solely due to missing values.

In [14]:
# Outlier filtering
df_clean = df_clean[
    (df_clean["trip_distance"] < 100) &
    (df_clean["fare_amount"] < 500)
]

### Outlier Filtering

We apply outlier filters to remove extreme values likely caused by recording errors:

- Trip distance more than 100 miles
- Fare amount more than $500

In [16]:
# Schema Standardization
df_clean.columns = (
    df_clean.columns
    .str.lower()
    .str.replace(" ", "_")
)

In [19]:
# Post-Cleaning validation
print("Rows before cleaning:", len(df))
print("Rows after cleaning:", len(df_clean))
print("Dropped rows:", len(df) - len(df_clean))

print("NA values:")
print(df_clean.isna().sum())
df_clean.describe()

Rows before cleaning: 22519712
Rows after cleaning: 21903044
Dropped rows: 616668
NA values:
vendorid                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
ratecodeid               0
store_and_fwd_flag       0
pulocationid             0
dolocationid             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
source_file              0
trip_duration_min        0
dtype: int64


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration_min
count,2.190304e+07,21903044,21903044,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07,2.190304e+07
mean,1.646588e+00,2019-02-15 07:39:05.272758016,2019-02-15 07:52:42.906417920,1.598944e+00,2.912673e+00,1.051840e+00,1.635959e+02,1.618016e+02,1.274207e+00,1.258252e+01,8.793517e-01,4.980721e-01,2.062933e+00,3.393932e-01,2.999816e-01,1.760019e+01,1.480001e+00,1.362723e+01
min,1.000000e+00,2019-01-01 00:00:00,2019-01-01 00:01:33,1.000000e+00,1.000000e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,-6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.100000e-01,0.000000e+00,1.666667e-02
25%,1.000000e+00,2019-01-24 10:26:44,2019-01-24 10:43:24,1.000000e+00,9.600000e-01,1.000000e+00,1.140000e+02,1.120000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.000000e+01,0.000000e+00,6.433333e+00
50%,2.000000e+00,2019-02-14 21:10:35,2019-02-14 21:24:42.500000,1.000000e+00,1.600000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.700000e+00,0.000000e+00,3.000000e-01,1.355000e+01,2.500000e+00,1.068333e+01
75%,2.000000e+00,2019-03-09 12:52:24,2019-03-09 13:05:52,2.000000e+00,2.940000e+00,1.000000e+00,2.330000e+02,2.340000e+02,2.000000e+00,1.400000e+01,1.000000e+00,5.000000e-01,2.750000e+00,0.000000e+00,3.000000e-01,1.896000e+01,2.500000e+00,1.741667e+01
max,4.000000e+00,2019-03-31 23:59:57,2019-04-01 00:46:12,6.000000e+00,9.920000e+01,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,4.680000e+02,1.850000e+01,6.080000e+01,7.872500e+02,3.288000e+03,6.000000e-01,3.345300e+03,2.750000e+00,2.399333e+02
std,5.243316e-01,NaN,NaN,1.220717e+00,3.788318e+00,4.922610e-01,6.603537e+01,7.019011e+01,4.600162e-01,1.106150e+01,1.136347e+00,3.929350e-02,2.560341e+00,1.795537e+00,2.351680e-03,1.380448e+01,1.228652e+00,1.069030e+01


### Post-Cleaning Validation

After cleaning:

- Verify that no critical columns contain missing values.
- Ensure that all trips satisfy business and physical constraints.
- Log the number of rows removed to quantify data quality impact.

This dataset is now suitable for feature engineering and modeling.

In [20]:
# Save clean snapshot
df_clean.to_parquet(CLEAN_PATH, index=False, engine="fastparquet")

## Cleaning Summary

- Raw rows: 22,519,712  
- Clean rows: 21,903,044  
- Dropped rows: 616,668 (~2.7%)

The cleaned dataset contains no missing values and satisfies all validity rules, making it suitable for feature engineering and modeling.